# YData Quality - Labelling (Numerical) Tutorial
Time-to-Value: 4 minutes

This notebook provides a tutorial for the ydata_quality package funcionality for numerical labels.

**Structure:**

1. Load dataset
2. Corrupt the dataset
2. Instantiate the Data Quality engine
3. Run the quality checks
4. Assess the warnings
5. (Extra) Detailed overview

In [1]:
# Update the imports
import statsmodels.api as sm
from ydata_quality.labelling import LabelInspector

## Load the example dataset
We will use a dataset available from the statsmodels package.

In [2]:
df = sm.datasets.get_rdataset('Guerry', 'HistData').data

In [3]:
df.head()

,dept,Region,Department,Crime_pers,Crime_prop,Literacy,Donations,Infants,Suicides,MainCity,...,Crime_parents,Infanticide,Donation_clergy,Lottery,Desertion,Instruction,Prostitutes,Distance,Area,Pop1831
0,1,E,Ain,28870,15890,37,5098,33120,35039,2:Med,...,71,60,69,41,55,46,13,218.372,5762,346.03
1,2,N,Aisne,26226,5521,51,8901,14572,12831,2:Med,...,4,82,36,38,82,24,327,65.945,7369,513.00
2,3,C,Allier,26747,7925,13,10973,17044,114121,2:Med,...,46,42,76,66,16,85,34,161.927,7340,298.26
3,4,E,Basses-Alpes,12935,7289,46,2733,23018,14238,1:Sm,...,70,12,37,80,32,29,2,351.399,6925,155.90
4,5,E,Hautes-Alpes,17488,8174,69,6962,23076,16171,1:Sm,...,22,23,64,79,35,7,1,320.280,5549,129.10


## Corrupt the dataset
Records without label and outlier labels are added to the dataset.

In [4]:
label = 'Donations'

df.loc[0, label] = None
df.loc[1, label] = 50000

## Create the engine
Each engine contains the checks and tests for each suite. To create a Label Inspector, you provide:
- df: target DataFrame, for which we will run the test suite
- label: name of the column to be used as label, in this case it points to a numerical label!

In [5]:
li = LabelInspector(df=df, label='Donations')

### Full Evaluation
The easiest way to assess the data quality analysis is to run `.evaluate()` which returns a list of warnings for each quality check. 

In [6]:
results = li.evaluate()

[TEST NORMALITY] It was not possible to normalize the label values. See the warning message for additional context.


## Check the status
After running the data quality checks, you can check the warnings for each individual test. The warnings are a dictionary of {test: result}.

In [7]:
li.report()

Warnings count by priority:
	Priority 1: 2 warning(s)
	Priority 2: 1 warning(s)
	TOTAL: 3 warning(s)
List of warnings sorted by priority:
	[TEST NORMALITY] The label distribution failed to pass a normality test as-is and following a battery of transforms. It is possible that the data originates from an exotic distribution, there is heavy outlier presence or it is multimodal. Addressing this issue might prove critical for regressor performance. (Priority 1: heavy impact expected)
	[MISSING LABELS] Found 1 instances with missing labels. (Priority 1: heavy impact expected)
	[OUTLIER DETECTION] Found 3 potential outliers across the full dataset. A distance bigger than 3.0 standard deviations of intra-cluster distances to the respective centroids was used to define the potential outliers. (Priority 2: usage allowed, limited human intelligibility)


### Quality Warning

In [8]:
# Get a sample warning
sample_warning = li.get_warnings()[1]

In [9]:
# Check the details
sample_warning.test, sample_warning.description, sample_warning.priority

('Missing labels', 'Found 1 instances with missing labels.', <Priority.P1: 1>)

In [10]:
# Retrieve the relevant data from the warning
sample_warning_data = sample_warning.data

## Full Test Suite
In this section, you will find a detailed overview of the available tests in the labelling module of ydata_quality.

### Missing Labels

Return records with a missing label.

In [11]:
li.missing_labels()

,dept,Region,Department,Crime_pers,Crime_prop,Literacy,Donations,Infants,Suicides,MainCity,...,Crime_parents,Infanticide,Donation_clergy,Lottery,Desertion,Instruction,Prostitutes,Distance,Area,Pop1831
0,1,E,Ain,28870,15890,37,NaN,33120,35039,2:Med,...,71,60,69,41,55,46,13,218.372,5762,346.03


### Outlier Detection

Get a dictionary of all potential outliers obtained over the full dataset (see example 1) or across clusters on the label feature (see example 2 below). Outliers are defined based on a threshold of standard deviations of distances to the median of the dataset or of the cluster (see example 2).

In [12]:
# Example 1
li.outlier_detection(th=3)  # The top 3 valued labels were flagged as potential outliers

{'full_dataset':     dept Region Department  Crime_pers  Crime_prop  Literacy  Donations  \
 1      2      N      Aisne       26226        5521        51    50000.0   
 12    14      N   Calvados       17577        4500        52    27830.0   
 85   200    NaN      Corse        2199        4589        49    37015.0   
 
     Infants  Suicides MainCity  ...  Crime_parents  Infanticide  \
 1     14572     12831    2:Med  ...              4           82   
 12     8983     31807    2:Med  ...             57           56   
 85    24743     37016    2:Med  ...             81            2   
 
     Donation_clergy  Lottery  Desertion  Instruction  Prostitutes  Distance  \
 1                36       38         82           24          327    65.945   
 12               11       13         12           22          194   117.487   
 85               84       83          9           25            1   539.213   
 
     Area  Pop1831  
 1   7369   513.00  
 12  5548   494.70  
 85  8680   195.41 

In [13]:
# Example 2
li.outlier_detection(th=3, use_clusters=True)  # Only the highest donation was flagged as a potential outlier

{3:    dept Region Department  Crime_pers  Crime_prop  Literacy  Donations  \
 1     2      N      Aisne       26226        5521        51    50000.0   
 
    Infants  Suicides MainCity  ...  Crime_parents  Infanticide  \
 1    14572     12831    2:Med  ...              4           82   
 
    Donation_clergy  Lottery  Desertion  Instruction  Prostitutes  Distance  \
 1               36       38         82           24          327    65.945   
 
    Area  Pop1831  
 1  7369    513.0  
 
 [1 rows x 23 columns]}

### Normality Test

Test the labels for normality (test if the data might originate from a normal distribution with a pre-specified confidence).
If the test fails apply a battery of transforms to the label and repeat the test after each transform.
Raise a low priority warning if a transform is required, raise a high priority warning if the transforms did not result in positive normality test.

In [14]:
li.test_normality()

[TEST NORMALITY] It was not possible to normalize the label values. See the warning message for additional context.
